In [505]:
import pandas as pd
import numpy as np
import nltk.corpus
import spacy
import re
from spacy_langdetect import LanguageDetector
#from autocorrect import Speller
#spell = Speller()
from profanity_filter import ProfanityFilter
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True) #2
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\Charlie
[nltk_data]     Lu\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [72]:
DATA = 'C:/Users/Charlie Lu/Desktop/Sentiment Analysis/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/Data/crafts.csv'

In [115]:
data = pd.read_csv(DATA,sep=",", encoding='cp1252',on_bad_lines='skip')
data_first = pd.read_csv(DATA,sep=",", encoding='cp1252',on_bad_lines='skip')[['Column1', 'Column2', 'Column3']]
data_first

,Column1,Column2,Column3
0,NaN,0,1
1,0.0,This octopus candle holder that my sister hand...,https://i.redd.it/mg4ge4iqymlz.jpg
2,1.0,My Hand-Painted Food Rocks,https://i.redd.it/x4unaiiccoq41.jpg
3,2.0,Trying to keep my kids busy! Painters tape + w...,https://i.redd.it/jgnnmda3x9q41.jpg
4,3.0,I started this before quarantine and have now ...,https://i.redd.it/socv1a1ttcv41.png
...,...,...,...
650,649.0,Wife made this Quilling art for a client on re...,https://i.redd.it/psu7ye81a1a51.jpg
651,650.0,Made a pillow,https://i.redd.it/m0aqqk74pe651.jpg
652,651.0,I crocheted a Chill Pill!,https://i.redd.it/6voptma7mnf21.jpg
653,652.0,I love making one of a kind kids clothes!,https://i.redd.it/fywgfrmh8fe21.jpg


In [364]:
data_subset = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
data_compare = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
data_subset

,Column4,Column5,Column6,Column7,Column8,Column9,Column10,Column11,Column12,Column13,...,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51,Column52,Column53
0,2,3,4,5,6,7,8,9,10,11,...,42,43,44,45,46,47,48,49,50,51
1,"This is amazing, does she have a website?",Incredible how he manages to stay still for so...,I love how pleased he looks with himself. \r\...,Your sister is a wizard,Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fh...,[removed],[Those eyes bring back fond memories!](https:/...,Shut up and take my money!!,This was handmade by my oldest daughter. She d...,I need this for Hanukkah. This would make an a...,...,It looks so happy to be useful by holding the ...,I've always preferred [this one](https://i.img...,I think your sister might be into tentacle porn.,"I can't believe she made this by hand, without...",pottery CLASS? is she the teacher?\r\n\r\ncall...,Absolutely phenomenal. Would love to see her p...,But what about your sister wives?,Hey I saw your sister A post this on Facebook ...,Yikes!! My comment came out wacky! My daughter...,Contact her at Nnyl_ahsatan1111
2,Since I saw someone else posting this the othe...,You should post this at r/forbiddensnacks,These are amazing! So much attention to detail,"Wow, these are incredible! What got you starte...",Oh wow- I might have commented on that complim...,Stoner food?\r\nEdit: came back to say I love ...,These are awesome. Also I love that duff and c...,"Great work Jen, you are a true artist!!!","These are great! For the letters, do you use p...",You are so skilled! Do you hide the rocks in p...,...,Fluff is my fave,I love this so much,Omg! This is really good! Now I want to start ...,I would break all my teeth. Just one after ano...,It truly is super sharp!,This is adorably brilliant!,"This would make a great poster,",that bologna rock lookin like a SNACK,Wow! These are fantastic! And they way this ph...,How cool!
3,This is so freaking cool! I'm tempted to do th...,You are a capital G Genius!!\r\n\r\nI don't ha...,Any kind of markers?\r\nSupervisor of a specia...,This is such a good idea for the little ones. ...,The shadows are super pretty but I would be mi...,I love this! And I just looked up “black paint...,I swear my first thought was “I bet that marke...,Just wow! Love the creativity!,I saw this on my neighbors window yesterday on...,Genius!! And it looks so pretty!,...,This makes more sense! Someone on my facebook ...,This is awesome!!!,Such a cool idea.,It’s so lovely!,"It doesn't have to be pretty, to be beautiful....",That is quite beautiful :),Brilliant!!!,So cool!,Excuse me while I transform my house.,Absolutely loving the idea! And it's so beauti...
4,You did an amazing job!!!,That's so amazing! Everything looks so accurat...,It's an heirloom now!,I was wondering where you got the pattern from...,WHAT THE HECK! this is amazing! i’m so impress...,Amazing dedication to finishing it!! Your hand...,Oh I think you missed a thread!! LOL....kiddin...,At first I thought “ that’s one hell of a nose...,“ITS TERRIBLE”\r\n\r\n..said no one ever,"Looks amazing, you did a great job. How about ...",...,WOWOWOWOWOW!!!,This is literally amazing! One of the most bea...,This is insanely beautiful!,Wow!,That's absolutely incredible! What an outstand...,That monkey finna get got when he gets his ass...,That is AMAZING!,WOW amazing. Look at all those colours... what...,"That is beautiful, I would buy",This is amazing! You are incredibly talented! ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,It's stunning,Lovely! Please share on r/quilling too!,"Your wife is literally SO talented, I loved he...",WOW! So amazing! Does she sell her work anywhe...,This is really beautiful. How much does someth...,Beautiful!,Impressive,She did an amazing job!!!!,Amazing!! I know how much time & patience that...,That is beautiful - thanks for sharing!,...,Thank you so much for the love guys.. a specia...,NaN,NaN,NaN,NaN,NaN,NaN

In [511]:
#Removes emoticons
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

#removes characters that appear more than twice ex: booo -> boo
keep_list = ['s','e','t','f','l','m','o','p']
def remove_dupes(s):
    ans = ""
    seen = ''
    i = 0
    while i != (len(s)-1):
        if s[i] in keep_list and s[i] != seen:
            if s[i] == s[i+1]:
                seen = s[i]
                ans+= s[i]
                ans+= s[i+1]
        if s[i] != seen and s[i+1] != s[i]:
            seen = s[i]
            ans += s[i]
            i += 1
        else:
            i += 1
    ans += s[len(s)-1]
    return ans

#language detection:

def language(text):
    doc = nlp(text)
    detect_language = doc._.language #4
    lang = detect_language['language']
    if lang == 'en':
        return True
    else:
        return False

#removes gibberish (Doesn't work, removes lots of real words as well)
'''
words = set(nltk.corpus.words.words())
def clean(sent):
    return " ".join(w for w in nltk.wordpunct_tokenize(sent) \
     if w.lower() in words or not w.isalpha())

text = 'building fence good mighty banana wiugSFNDnrgkdf help'
print(clean(text))
'''

'\nwords = set(nltk.corpus.words.words())\ndef clean(sent):\n    return " ".join(w for w in nltk.wordpunct_tokenize(sent)      if w.lower() in words or not w.isalpha())\n\ntext = \'building fence good mighty banana wiugSFNDnrgkdf help\'\nprint(clean(text))\n'

3

In [521]:
#Subsets dataset and define filters
data_subset = data.drop(['Column1', 'Column2', 'Column3'], axis=1)
delete_list = ['[deleted]','[removed]']
sub_list = [r'r/\S+',r'\([^)]*\)',r'http\S+',r'[()]',r'[:]',r'[\([{})\]]']
pf = ProfanityFilter()

for i in range(len(data_subset)):
#for i in range(400,401):
    for j in range(len(data_subset.iloc[i])):
        text = data_subset.iloc[i][j]
        try:
            text = remove_emojis(text)
        except:
            continue
        text = remove_dupes(text)
        text = text.replace("\r", " ").replace("\n", " ").replace("\t", " ").replace("_x000D_", " ").strip()
        #text = spell(text)
        #text = clean(text)
        try:
            text = re.sub(r'\([^)]*\)', '', text)
            for sub in sub_list:
                text = re.sub(sub,'',text)
        except:
            continue
        text = pf.censor(text)
        if any(word in data_subset.iloc[i][j] for word in delete_list):
            data_subset.iloc[i][j] = np.nan
        data_subset['Column' + str(j+4)][i] = text
    print(i,"out of",len(data_subset))

0 out of 655
1 out of 655
2 out of 655
3 out of 655
4 out of 655
5 out of 655
6 out of 655
7 out of 655
8 out of 655
9 out of 655
10 out of 655
11 out of 655
12 out of 655
13 out of 655
14 out of 655
15 out of 655
16 out of 655
17 out of 655
18 out of 655
19 out of 655
20 out of 655
21 out of 655
22 out of 655
23 out of 655
24 out of 655
25 out of 655
26 out of 655
27 out of 655
28 out of 655
29 out of 655
30 out of 655
31 out of 655
32 out of 655
33 out of 655
34 out of 655
35 out of 655
36 out of 655
37 out of 655
38 out of 655
39 out of 655
40 out of 655
41 out of 655
42 out of 655
43 out of 655
44 out of 655
45 out of 655
46 out of 655
47 out of 655
48 out of 655
49 out of 655
50 out of 655
51 out of 655
52 out of 655
53 out of 655
54 out of 655
55 out of 655
56 out of 655
57 out of 655
58 out of 655
59 out of 655
60 out of 655
61 out of 655
62 out of 655
63 out of 655
64 out of 655
65 out of 655
66 out of 655
67 out of 655
68 out of 655
69 out of 655
70 out of 655
71 out of 655
72

554 out of 655
555 out of 655
556 out of 655
557 out of 655
558 out of 655
559 out of 655
560 out of 655
561 out of 655
562 out of 655
563 out of 655
564 out of 655
565 out of 655
566 out of 655
567 out of 655
568 out of 655
569 out of 655
570 out of 655
571 out of 655
572 out of 655
573 out of 655
574 out of 655
575 out of 655
576 out of 655
577 out of 655
578 out of 655
579 out of 655
580 out of 655
581 out of 655
582 out of 655
583 out of 655
584 out of 655
585 out of 655
586 out of 655
587 out of 655
588 out of 655
589 out of 655
590 out of 655
591 out of 655
592 out of 655
593 out of 655
594 out of 655
595 out of 655
596 out of 655
597 out of 655
598 out of 655
599 out of 655
600 out of 655
601 out of 655
602 out of 655
603 out of 655
604 out of 655
605 out of 655
606 out of 655
607 out of 655
608 out of 655
609 out of 655
610 out of 655
611 out of 655
612 out of 655
613 out of 655
614 out of 655
615 out of 655
616 out of 655
617 out of 655
618 out of 655
619 out of 655
620 out of

In [522]:
#Duplicate text removal testing
#text = 'r9h32w9EAJFIDOSNGFMAJIWEMFODimns hello fifwoeldjgm'
text = 'bluuuuuue abysssss, trrrrolllllllllllllllllll, heelllllpppp meeeee, yyellowww,,,, !!!!,#@$$$'
remove_dupes(text)
#len(text)
#language(text)

'blue abyss, troll, heellpp mee, yellow, !,#@$'

In [488]:
#compares raw vs filtered data

index = 400
for i in range(len(data_subset.iloc[index])):
    print ('old:',data_compare.iloc[index][i])
    print ('new:',data_subset.iloc[index][i])
    print(" ")
    


old: That is a lovely raku hippo ??.
new: That is a lovely raku hipo ?.
 
old: They call him moto moto
new: They call him moto moto
 
old: Did she paint it or create the whole piece? It’s so cute!
new: Did she paint it or create the whole piece? It’s so cute!
 
old: Aww, he is waving hello with his little foot.
new: Aw, he is waving hello with his little foot.
 
old: "William" from the Met, doing a little dance!
new: "William" from the Met, doing a little dance!
 
old: House hippo.
new: House hipo.
 
old: Is your mom Jeanette Alsing?
new: Is your mom Jeanette Alsing?
 
old: I love this. Your mom is very talented.
new: I love this. Your mom is very talented.
 
old: My dogs named hippo and he does this exact thing when he wants a belly rub.  Does your mom commission?  Feel free to PM me if she does.
new: My dogs named hipo and he does this exact thing when he wants a belly rub. Does your mom commission? Feel free to PM me if she does.
 
old: I want one! That is adorable!
new: I want one!

In [489]:
result = pd.concat([data_first, data_subset], axis=1, join='inner')
result

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column44,Column45,Column46,Column47,Column48,Column49,Column50,Column51,Column52,Column53
0,NaN,0,1,2,3,4,5,6,7,8,...,42,43,4,45,46,47,48,49,50,51
1,0.0,This octopus candle holder that my sister hand...,https://i.redd.it/mg4ge4iqymlz.jpg,"This is amazing, does she have a website?",Incredible how he manages to stay still for so...,I love how pleased he looks with himself. Be...,Your sister is a wizard,Ph'nglui mglw'nafh Cthulhu R'lyeh wgah'nagl fh...,removed,Those eyes bring back fond memories!,...,It looks so hapy to be useful by holding the c...,I've always prefered this one.,I think your sister might be into tentacle ****.,"I can't believe she made this by hand, without...",pottery CLAS? is she the teacher? calling b...,Absolutely phenomenal. Would love to see her p...,But what about your sister wives?,Hey I saw your sister A post this on Facebook ...,Yikes! My comment came out wacky! My daughter ...,Contact her at Nnyl_ahsatan1
2,1.0,My Hand-Painted Food Rocks,https://i.redd.it/x4unaiiccoq41.jpg,Since I saw someone else posting this the othe...,You should post this at,These are amazing! So much attention to detail,"Wow, these are incredible! What got you starte...",Oh wow- I might have commented on that complim...,Stoner food? Edit came back to say I love the...,These are awesome. Also I love that duff and c...,...,Fluff is my fave,I love this so much,Omg! This is really good! Now I want to start ...,I would break all my teeth. Just one after ano...,It truly is super sharp!,This is adorably brilliant!,"This would make a great poster,",that bologna rock lookin like a SNACK,Wow! These are fantastic! And they way this ph...,How cool!
3,2.0,Trying to keep my kids busy! Painters tape + w...,https://i.redd.it/jgnnmda3x9q41.jpg,This is so freaking cool! I'm tempted to do th...,You are a capital G Genius! I don't have li...,Any kind of markers? Supervisor of a special ...,This is such a good idea for the little ones. ...,The shadows are super pretty but I would be mi...,I love this! And I just looked up “black paint...,I swear my first thought was “I bet that marke...,...,This makes more sense! Someone on my facebook ...,This is awesome!,Such a cool idea.,It’s so lovely!,"It doesn't have to be pretty, to be beautiful....",That is quite beautiful,Brilliant!,So cool!,Excuse me while I transform my house.,Absolutely loving the idea! And it's so beauti...
4,3.0,I started this before quarantine and have now ...,https://i.redd.it/socv1a1ttcv41.png,You did an amazing job!,That's so amazing! Everything looks so acurate...,It's an heirloom now!,I was wondering where you got the pattern from...,WHAT THE HECK! this is amazing! i’m so impress...,Amazing dedication to finishing it! Your hands...,Oh I think you missed a thread! LOL.kiding! Th...,...,WOWOWOWOWOW!,This is literally amazing! One of the most bea...,This is insanely beautiful!,Wow!,That's absolutely incredible! What an outstand...,That monkey fina get got when he gets his *** ...,That is AMAZING!,WOW amazing. Look at all those colours. what a...,"That is beautiful, I would buy",This is amazing! You are incredibly talented! ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,649.0,Wife made this Quilling art for a client on re...,https://i.redd.it/psu7ye81a1a51.jpg,It's stuning,Lovely! Please share on too!,"Your wife is literally SO talented, I loved he...",WOW! So amazing! Does she sell her work anywhe...,This is really beautiful. How much does someth...,Beautiful!,Impressive,...,Thank you so much for the love guys. a special...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651,650.0,Made a pillow,https://i.redd.it/m0aqqk74pe651.jpg,It’s so pretty! It looks so soft too! What did...,Did you make this after watching Christine McC...,Love! But also that chai fabric color are beau...,I just watched this video yesterday that Chris...,Wow! What’s it made of? How long did 

In [490]:
result.to_csv(f'C:/Users/Charlie Lu/Desktop/Sentiment Analysis/BERT-Sentiment-Analysis-Reddit-API/DataCleaning/Data/crafts_test.csv', index=False)